**import dataset**

In [ ]:
!git clone https://github.com/elaamranisoufiane/Comp2018DatasetCleanISR.git

In [ ]:
# import numpy as np
# import tensorflow as tf
# from tensorflow.keras import backend as K

# from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
# from tensorflow.keras.models import Model
# from matplotlib import pyplot as plt
# import matplotlib.image as mpimg

# def iou(y_true, y_pred):
#     def f(y_true, y_pred):
#         intersection = (y_true * y_pred).sum()
#         union = y_true.sum() + y_pred.sum() - intersection
#         x = (intersection + 1e-15) / (union + 1e-15)
#         x = x.astype(np.float32)
#         return x
#     return tf.numpy_function(f, [y_true, y_pred], tf.float32)

# smooth = 1e-15
# def dice_coef(y_true, y_pred):
#     y_true = tf.keras.layers.Flatten()(y_true)
#     y_pred = tf.keras.layers.Flatten()(y_pred)
#     intersection = tf.reduce_sum(y_true * y_pred)
#     return (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)

# def dice_loss(y_true, y_pred):
#     return 1.0 - dice_coef(y_true, y_pred)

# from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, Conv2DTranspose, Concatenate, Input
# from tensorflow.keras.models import Model
# from tensorflow.keras.applications import ResNet50

# def conv_block(inputs, num_filters):
#     x = Conv2D(num_filters, 3, padding="same")(inputs)
#     x = BatchNormalization()(x)
#     x = Activation("relu")(x)

#     x = Conv2D(num_filters, 3, padding="same")(x)
#     x = BatchNormalization()(x)
#     x = Activation("relu")(x)

#     return x

# def decoder_block(inputs, skip_features, num_filters):
#     x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(inputs)
#     x = Concatenate()([x, skip_features])
#     x = conv_block(x, num_filters)
#     return x

# def build_resnet50_unet(input_shape):
#     """ Input layer """
#     inputs = Input(input_shape)

#     """ Pre-trained ResNet50 Model """
#     resnet50 = ResNet50(include_top=False, weights='imagenet', input_tensor=inputs)
    
#     # Extract the feature maps
#     s1 = resnet50.get_layer("conv1_relu").output           # 64
#     s2 = resnet50.get_layer("conv2_block3_out").output     # 256
#     s3 = resnet50.get_layer("conv3_block4_out").output     # 512
#     s4 = resnet50.get_layer("conv4_block6_out").output     # 1024
#     b1 = resnet50.get_layer("conv5_block3_out").output     # 2048

#     """ Decoder """
#     d1 = decoder_block(b1, s4, 512)
#     d2 = decoder_block(d1, s3, 256)
#     d3 = decoder_block(d2, s2, 128)
#     d4 = decoder_block(d3, s1, 64)
#     d5 = Conv2DTranspose(32, (2, 2), strides=2, padding="same")(d4)
#     d6 = conv_block(d5, 16)

#     """ Output layer """
#     outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(d6)

#     model = Model(inputs, outputs, name="ResNet50_UNET")
#     return model

# if __name__ == "__main__":
#     model = build_resnet50_unet((1024, 1024, 3))
#     model.summary()

 

# **##test##**

In [ ]:
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import numpy as np
import cv2
from glob import glob
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Recall, Precision
from sklearn.model_selection import KFold
from tqdm import tqdm
import pandas as pd

#import build_unet
#import dice_coef, iou
 
from tensorflow.keras.utils import CustomObjectScope
from sklearn.metrics import accuracy_score, f1_score, jaccard_score, precision_score, recall_score


# input image size
H = 1024
W = 1024

def iou(y_true, y_pred):
    def f(y_true, y_pred):
        intersection = (y_true * y_pred).sum()
        union = y_true.sum() + y_pred.sum() - intersection
        x = (intersection + 1e-15) / (union + 1e-15)
        x = x.astype(np.float32)
        return x
    return tf.numpy_function(f, [y_true, y_pred], tf.float32)

smooth = 1e-15
def dice_coef(y_true, y_pred):
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)

def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

# create directory and gives the path
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)


def shuffling(x, y):
    x, y = shuffle(x, y, random_state=42)


def read_image(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (W, H))
    x = x / 255.0
    x = x.astype(np.float32)
    return x


def read_mask(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    x = cv2.resize(x, (W, H))
    x = x / 255.0
    x = x.astype(np.float32)
    x = np.expand_dims(x, axis=-1)
    return x


def tf_parse(x, y):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        return x, y

    x, y = tf.numpy_function(_parse, [x, y], [tf.float32, tf.float32])
    x.set_shape([H, W, 3])
    y.set_shape([H, W, 1])
    return x, y



def tf_dataset(X, Y, batch=8):
    dataset = tf.data.Dataset.from_tensor_slices((X, Y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch)
    dataset = dataset.prefetch(10)
    return dataset

##########################################

def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)


def read_image1(path):
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (W, H))
    ori_x = x
    x = x / 255.0
    x = x.astype(np.float32)
    x = np.expand_dims(x, axis=0)  ## (1, 256, 256, 3)
    return ori_x, x


def read_mask1(path):
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    x = cv2.resize(x, (W, H))
    ori_x = x
    x = x / 255.0
    x = x > 0.5
    x = x.astype(np.int32)
    return ori_x, x

def save_result(ori_x, ori_y, y_pred, save_path):
    line = np.ones((H, 10, 3)) * 255

    ori_y = np.expand_dims(ori_y, axis=-1)  ## (256, 256, 1)
    ori_y = np.concatenate([ori_y, ori_y, ori_y], axis=-1)  ## (256, 256, 3)

    y_pred = np.expand_dims(y_pred, axis=-1)
    y_pred = np.concatenate([y_pred, y_pred, y_pred], axis=-1) * 255.0

    cat_images = np.concatenate([ori_x, line, ori_y, line, y_pred], axis=1)
    cv2.imwrite(save_path, cat_images)

def load_data(path, split=0.2):
    images = sorted(glob(os.path.join(path, "images", "*.jpg")))
    masks = sorted(glob(os.path.join(path, "Masks", "*.jpg")))
    size = int(len(images) * split)

    train_x, valid_x = train_test_split(images, test_size=size, random_state=42)
    train_y, valid_y = train_test_split(masks, test_size=size, random_state=42)

    train_x, test_x = train_test_split(train_x, test_size=size, random_state=42)
    train_y, test_y = train_test_split(train_y, test_size=size, random_state=42) 
    return (train_x, train_y), (valid_x, valid_y), (test_x, test_y)


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, CSVLogger
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Recall, Precision
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, f1_score, jaccard_score, recall_score, precision_score
from tqdm import tqdm
import pandas as pd
import os

# Ensure the required functions such as `create_dir`, `load_data`, `tf_dataset`, `read_image1`, `read_mask1`, and `save_result` are defined here or imported

def conv_block(inputs, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

def decoder_block(inputs, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(inputs)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

def build_resnet50_unet(input_shape):
    """ Input layer """
    inputs = Input(input_shape)

    """ Pre-trained ResNet50 Model """
    resnet50 = ResNet50(include_top=False, weights='imagenet', input_tensor=inputs)
    
    # Extract the feature maps
    s1 = resnet50.get_layer("conv1_relu").output           # 64
    s2 = resnet50.get_layer("conv2_block3_out").output     # 256
    s3 = resnet50.get_layer("conv3_block4_out").output     # 512
    s4 = resnet50.get_layer("conv4_block6_out").output     # 1024
    b1 = resnet50.get_layer("conv5_block3_out").output     # 2048

    """ Decoder """
    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)
    d5 = Conv2DTranspose(32, (2, 2), strides=2, padding="same")(d4)
    d6 = conv_block(d5, 16)

    """ Output layer """
    outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(d6)

    model = Model(inputs, outputs, name="ResNet50_UNET")
    return model

if __name__ == "__main__":
    """ Seeding """
    np.random.seed(42)
    tf.random.set_seed(42)

    """ Create directories """
    def create_dir(path):
        if not os.path.exists(path):
            os.makedirs(path)
            
    create_dir('files')
    create_dir("results")

    """ Hyperparameters """
    batch_size = 4
    lr = 1e-4  ## 0.0001
    num_epochs = 40
    model_path = "files/model.h5"
    csv_path = "files/data.csv"
    dataset_path = "./Comp2018DatasetCleanISR"
    
    """ Dataset """
    (train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_data(dataset_path)
    
    print(f"Train: {len(train_x)} - {len(train_y)}")
    print(f"Valid: {len(valid_x)} - {len(valid_y)}")
    print(f"Test: {len(test_x)} - {len(test_y)}")
    
    acc_per_fold = []
    loss_per_fold = []
    f1_value_per_fold = []
    jac_value_per_fold = []
    recall_value_per_fold = []
    precision_value_per_fold = []
    
    num_folds = 4
    # Define the K-fold Cross Validator
    kfold = KFold(n_splits=num_folds, shuffle=True)
    
    # K-fold Cross Validation model evaluation
    fold_no = 1
    for train, test in kfold.split(train_x, train_y):
        print(f'Training for fold {fold_no} ...')
        
        train_x1 = [train_x[i] for i in train]
        train_y1 = [train_y[i] for i in train]
        test_x1 = [train_x[i] for i in test]
        test_y1 = [train_y[i] for i in test]

        print('------------------------------------------------------------------------')
        print(f'Training for fold {fold_no} ...')
        print(f"Train: {len(train_x1)} - {len(train_y1)}")
        print(f"Valid: {len(valid_x)} - {len(valid_y)}")
        print(f"Test: {len(test_x1)} - {len(test_y1)}")
        
        train_dataset = tf_dataset(train_x1, train_y1, batch=batch_size)
        valid_dataset = tf_dataset(valid_x, valid_y, batch=batch_size)
        train_steps = (len(train_x1) // batch_size)
        valid_steps = (len(valid_x) // batch_size)
        if len(train_x1) % batch_size != 0:
            train_steps += 1

        for x, y in valid_dataset:
            print(x.shape, y.shape)
            break

        """ Build the model with ResNet50 U-Net network architecture """
        model = build_resnet50_unet((1024, 1024, 3))
        metrics = ["accuracy", dice_coef, iou, Recall(), Precision()]
        model.compile(loss="binary_crossentropy", optimizer=Adam(lr), metrics=metrics)
        
        callbacks = [
            ModelCheckpoint(model_path, verbose=1, save_best_only=True),
            ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-7, verbose=1),
            CSVLogger(csv_path),
        ]
        
        history = model.fit(
            train_dataset,
            epochs=num_epochs,
            validation_data=valid_dataset,
            steps_per_epoch=train_steps,
            validation_steps=valid_steps,
            callbacks=callbacks,
            shuffle=False
        )
        fold_no = fold_no + 1

        """ Load Model """
        with CustomObjectScope({'iou': iou, 'dice_coef': dice_coef}):
            model = tf.keras.models.load_model(model_path)
        
        """ Prediction and metrics values """
        SCORE = []
        for x, y in tqdm(zip(test_x1, test_y1), total=len(test_x1)):
            name = x.split("/")[-1]
            
            """ Reading the image and mask """
            ori_x, x = read_image1(x)
            ori_y, y = read_mask1(y)
            
            """ Prediction """
            y_pred = model.predict(x)[0] > 0.5
            y_pred = np.squeeze(y_pred, axis=-1)
            y_pred = y_pred.astype(np.int32)
            
            save_path = f"results/{name}"
            save_result(ori_x, ori_y, y_pred, save_path)
            
            """ Flattening the numpy arrays. """
            y = y.flatten()
            y_pred = y_pred.flatten()
            
            """ Calculating metrics values """
            acc_value = accuracy_score(y, y_pred)
            f1_value = f1_score(y, y_pred, labels=[0, 1], average="binary")
            jac_value = jaccard_score(y, y_pred, labels=[0, 1], average="binary")
            recall_value = recall_score(y, y_pred, labels=[0, 1], average="binary")
            precision_value = precision_score(y, y_pred, labels=[0, 1], average="binary")
            SCORE.append([name, acc_value, f1_value, jac_value, recall_value, precision_value])

        """ Metrics values """
        score = [s[1:] for s in SCORE]
        score = np.mean(score, axis=0)
        print(f"Accuracy: {score[0]:0.5f}")
        print(f"F1: {score[1]:0.5f}")
        print(f"Jaccard: {score[2]:0.5f}")
        print(f"Recall: {score[3]:0.5f}")
        print(f"Precision: {score[4]:0.5f}")
        
        """ Save the results """
        acc_per_fold.append(score[0])
        f1_value_per_fold.append(score[1])
        jac_value_per_fold.append(score[2])
        recall_value_per_fold.append(score[3])
        precision_value_per_fold.append(score[4])
        
        """ Saving all the results """
        df = pd.DataFrame(SCORE, columns=["Image", "Accuracy", "F1", "Jaccard", "Recall", "Precision"])
        df.to_csv("files/score.csv")

    """ Final metrics """
    print(f"Accuracy: {acc_per_fold}")
    print(f"F1: {f1_value_per_fold}")
    print(f"Jaccard: {jac_value_per_fold}")
    print(f"Recall: {recall_value_per_fold}")
    print(f"Precision: {precision_value_per_fold}")


In [ ]:
import numpy as np
print(f"Accuracy: {np.ceil(np.mean(acc_per_fold)*10000)/100}%")
print(f"F1: {np.ceil(np.mean(f1_value_per_fold)*10000)/100}%")
print(f"Jaccard: {np.ceil(np.mean(jac_value_per_fold)*10000)/100}%")
print(f"Recall: {np.ceil(np.mean(recall_value_per_fold)*10000)/100}%")
print(f"Precision: {np.ceil(np.mean(precision_value_per_fold)*10000)/100}%")

In [ ]:
import matplotlib.pyplot as plt
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.savefig('loss.png')
plt.show()
#plt.imshow(mpimg.imread('./loss.png'))

accu = history.history['accuracy']
val_accu = history.history['val_accuracy']
epochs = range(1, len(accu) + 1)
plt.plot(epochs, accu, 'r', label='accuracy') 
plt.plot(epochs, val_accu, 'b', label='val_accuracy') 
plt.title('accuracy')
plt.xlabel('Epochs')
plt.ylabel('accuracy')
plt.legend()
plt.savefig('accuracy.png')
plt.show()
#plt.imshow(mpimg.imread('./accuracy.png'))

**Testing**

In [ ]:
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import numpy as np
import cv2
import pandas as pd
from glob import glob
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.utils import CustomObjectScope
from sklearn.metrics import accuracy_score, f1_score, jaccard_score, precision_score, recall_score
#import dice_coef, iou
#import load_data

H = 1024
W = 1024


def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)


def read_image(path):
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (W, H))
    ori_x = x
    x = x / 255.0
    x = x.astype(np.float32)
    x = np.expand_dims(x, axis=0)  ## (1, 256, 256, 3)
    return ori_x, x


def read_mask(path):
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    x = cv2.resize(x, (W, H))
    ori_x = x
    x = x / 255.0
    x = x > 0.5
    x = x.astype(np.int32)
    return ori_x, x


def save_result(ori_x, ori_y, y_pred, save_path):
    line = np.ones((H, 10, 3)) * 255

    ori_y = np.expand_dims(ori_y, axis=-1)  ## (256, 256, 1)
    ori_y = np.concatenate([ori_y, ori_y, ori_y], axis=-1)  ## (256, 256, 3)

    y_pred = np.expand_dims(y_pred, axis=-1)
    y_pred = np.concatenate([y_pred, y_pred, y_pred], axis=-1) * 255.0

    cat_images = np.concatenate([ori_x, line, ori_y, line, y_pred], axis=1)
    cv2.imwrite(save_path, cat_images)


if __name__ == "__main__":
    create_dir("results")

    """ Load Model """
    with CustomObjectScope({'iou': iou, 'dice_coef': dice_coef}):
        model = tf.keras.models.load_model("files/model.h5")
        #model.summary()

    """ Dataset """
    dataset_path = "./Comp2018DatasetCleanISR"
    #dataset_path = "/imagesToAugment/images"
    (train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_data(dataset_path)

    """ Prediction and metrics values """
    SCORE = []
    for x, y in tqdm(zip(test_x, test_y), total=len(test_x)):
        name = x.split("/")[-1]

        """ Reading the image and mask """
        ori_x, x = read_image(x)
        ori_y, y = read_mask(y)

        """ Prediction """
        y_pred = model.predict(x)[0] > 0.5
        y_pred = np.squeeze(y_pred, axis=-1)
        y_pred = y_pred.astype(np.int32)

        save_path = f"results/{name}"
        save_result(ori_x, ori_y, y_pred, save_path)

        """ Flattening the numpy arrays. """
        y = y.flatten()
        y_pred = y_pred.flatten()

        """ Calculating metrics values """
        acc_value = accuracy_score(y, y_pred)
        f1_value = f1_score(y, y_pred, labels=[0, 1], average="binary")
        jac_value = jaccard_score(y, y_pred, labels=[0, 1], average="binary")
        recall_value = recall_score(y, y_pred, labels=[0, 1], average="binary")
        precision_value = precision_score(y, y_pred, labels=[0, 1], average="binary")
        SCORE.append([name, acc_value, f1_value, jac_value, recall_value, precision_value])

    """ Metrics values """
    score = [s[1:] for s in SCORE]
    score = np.mean(score, axis=0)
    print(f"Accuracy: {score[0]:0.5f}")
    print(f"F1: {score[1]:0.5f}")
    print(f"Jaccard: {score[2]:0.5f}")
    print(f"Recall: {score[3]:0.5f}")
    print(f"Precision: {score[4]:0.5f}")

    """ Saving all the results """
    df = pd.DataFrame(SCORE, columns=["Image", "Accuracy", "F1", "Jaccard", "Recall", "Precision"])
    df.to_csv("files/score.csv")

In [ ]:
import cv2
import glob
import matplotlib.pyplot as plt
    
path="./results/*.jpg"

images=[cv2.imread(image) for image in glob.glob(path)]
fig=plt.figure()
for i in range(len(images)):
    #plt.subplot(5,5,i+1)
    plt.imshow(images[i])
    plt.show()

**Zip the result file**

In [ ]:
!zip -r ./file.zip ./results 

**download te results file**

In [ ]:
from IPython.display import FileLink
FileLink(r'file.zip')

# **Test**